# Usage of shared pointers

Shared pointers are useful if many clients need to access the same resource. When a client goes out-of-scope it disconnects from the shared resource. The last pointer deletes the object.

Example class for payload as shared resource: It just visualizes constructors / destructors being called.

In [ ]:
class Payload {
  public:
    Payload() { std::cout << "Payload constructor called" << std::endl; }
    Payload(const Payload &ref) { std::cout << "Payload copy constructor called" << std::endl; }
    Payload &operator=(const Payload &ref) { std::cout << "Payload assignment operator called" << std::endl; return *this; }
    ~Payload() { std::cout << "Payload destructor called" << std::endl; }
};

Creating a client class: The class has a shared pointer to a payload instance. Sharing the payload happens when the copy constructor is called.

In [ ]:
class Client {
    std::shared_ptr<Payload> fPayload;
    
    public:
    Client() : fPayload() { std::cout << "Client constructor called" << std::endl; }
    Client(Payload *payload) : fPayload(payload) { std::cout << "Client constructor with payload called: Currently " << fPayload.use_count() << " clients accessing the payload" << std::endl; }
    Client(const Client &ref) : fPayload(ref.fPayload) { std::cout << "Client copy constructor called: Currently " << fPayload.use_count() << " clients accessing the payload" << std::endl; }
    Client &operator=(const Client &ref) {
        if(&ref != this) {
            fPayload = ref.fPayload;
            std::cout << "Client assignment operator called: Currently " << fPayload.use_count() << " clients accessing the payload" << std::endl;
        }
        return *this;
    }
    ~Client() { std::cout << "Destructor called : Currently " << fPayload.use_count() << " clients accessing the payload (before delete)" << std::endl; }
};

Creating the first client where we set the payload. It will set the ref count to 1.

In [ ]:
Client * base = new Client(new Payload);

Create other clients related from the base client. We now have multiple clients accessing the same resource ...

In [ ]:
std::vector<Client *> others;
for(int i = 0; i < 10; i++) {
    others.push_back(new Client(*base));
}

We now delete all the clients related from the base client. The reference count is always obtained before the client shared pointer is deleted.

In [ ]:
for(auto o : others) delete o;

Deleting the last client instance. It will delete the payload as well as it is the last instance accessing it.

In [ ]:
delete base;